<a href="https://colab.research.google.com/github/shub2022/Text_classifier_Healthcare/blob/main/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai.text.all import *

from google.colab import drive
import os

drive.mount('/content/drive')
path = '/content/drive/MyDrive/Kaggle_int_hc'
os.listdir(path)


Mounted at /content/drive


['train', 'valid']

In [2]:
# DataBlock is an API which is used for building a dataloader

# https://docs.fast.ai/tutorial.datablock.html

dblock = DataBlock(blocks=(TextBlock.from_folder(path=path), CategoryBlock),
                   get_items=partial(get_text_files, folders=['train', 'valid']),
                   get_y=parent_label,
                   splitter=GrandparentSplitter(valid_name='valid'))

In [3]:
# Now creating a dataloader using dblock created above

dls = dblock.dataloaders(source=path, bs=32)

In [5]:
len(dls.train), len(dls.valid), len(dls.train.dataset), len(dls.valid.dataset)

(84, 22, 2707, 677)

In [39]:
dls.show_batch()

,text,category
0,"xxbos xxmaj chiropractic xxup ime with old files review . xxmaj detailed xxmaj thoracic xxmaj spine xxmaj examination . transcription : xxup date xxup of xxup injury : xxmaj october 4 , 2 xxrep 3 0 , date xxup of xxup examination : xxmaj september 5 , 2003,examining xxup physician : x xxup y , md , prior to the beginning of the examination , it is explained to the examinee that this examination is intended for evaluative purposes only , and that it is not intended to constitute a general medical examination . xxmaj it is explained to the examinee that the traditional doctor - patient relationship does not apply to this examination , and that a written report will be provided to the agency requesting this examination . xxmaj it has also been emphasized to the examinee that he should not attempt any physical activity beyond his tolerance",IME-QME-Work Comp etc.
1,"xxbos xxmaj chiropractic xxup ime with old files review . xxmaj detailed xxmaj thoracic xxmaj spine xxmaj examination . transcription : xxup date xxup of xxup injury : xxmaj october 4 , 2 xxrep 3 0 , date xxup of xxup examination : xxmaj september 5 , 2003,examining xxup physician : x xxup y , md , prior to the beginning of the examination , it is explained to the examinee that this examination is intended for evaluative purposes only , and that it is not intended to constitute a general medical examination . xxmaj it is explained to the examinee that the traditional doctor - patient relationship does not apply to this examination , and that a written report will be provided to the agency requesting this examination . xxmaj it has also been emphasized to the examinee that he should not attempt any physical activity beyond his tolerance",Chiropractic
2,"xxbos xxmaj pain management for post - laminectomy low back syndrome and radiculopathy . transcription : xxmaj mr . xxup xyz forgot his hearing aids at home today and is severely hearing impaired and most of the interview had to be conducted with me xxunk at him at the top of my voice . xxmaj for all these reasons , this was not really under the best circumstances and i had to xxunk the amount of time i spent trying to get a history because of the physical effort required in xxunk information from this patient . xxmaj the patient was seen late because he had not filled in the patient questionnaire . xxmaj to xxunk the history here , xxmaj mr . xxup xyz who is not very clear on events from the past , sustained a work - related injury some time in 1998 . xxmaj at that",Chiropractic
3,"xxbos xxmaj the patient is with multiple neurologic and nonneurologic symptoms including numbness , gait instability , decreased dexterity of his arms and general fatigue . xxmaj his neurological examination is notable for sensory loss in a length - dependent fashion in his feet and legs with scant fasciculations in his calves . transcription : xxup history xxup of xxup present xxup illness : , xxmaj the patient is a 63 - year - old left - handed gentleman who presents for further evaluation of multiple neurological symptoms . i asked him to discuss each symptom individually as he had a very hard time describing the nature of his problems . xxmaj he first mentioned that he has neck pain . xxmaj he states that he has had this for at least 15 years . xxmaj it is worse with movement . xxmaj it has progressed very slowly over the",Consult - History and Phy.
4,"xxbos xxmaj autopsy of a white female who died of acute combined drug intoxication . transcription : xxup clothing : , xxmaj the body is xxunk in a light green hospital gown , which is intact , dry and clean . xxmaj there is no xxunk present . ,external xxup examination : , xxmaj the body is that of a well - developed , well - nourished white woman appearing the offered age of 39 years . xxmaj the body measures 71 inches and weighs 178 pounds . ,the xxunk body is well preserved and cool to touch due to xxunk . xxmaj xxunk mortis is fully developed in the major muscle group

In [4]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [5]:
# Fine tuning the pre-trained AWD LSTM model using the training data

learn.fine_tune(4, 1e-2)


epoch,train_loss,valid_loss,accuracy,time
0,2.573861,2.186901,0.342688,03:17


epoch,train_loss,valid_loss,accuracy,time
0,2.012829,2.118516,0.322009,01:23
1,1.870983,1.934213,0.336780,01:23
2,1.638653,1.883097,0.301329,01:23
3,1.442583,1.861280,0.305761,01:23


Performance is pretty bad

In fastai, the term "0th epoch" refers to the initial phase of training where the model is exposed to the data for the first time, but no actual training updates are performed. During this phase, the model computes the necessary statistics for normalization layers (like batch normalization) and might perform other tasks such as creating an adaptive learning rate schedule.

The "0th epoch" is a term used in the context of transfer learning and fine-tuning. When you fine-tune a pre-trained model on a new task, the first epoch is often treated differently from subsequent epochs to ensure a smooth transition from the pre-trained weights to the task-specific weights.

Here is a typical workflow during the "0th epoch" in fastai:

Model Initialization:

The pre-trained weights are loaded into the model.
Batch Normalization Statistics:

The model processes a few batches from the training dataset to compute batch normalization statistics. This is crucial for normalizing the input data during training.
Learning Rate Schedule Initialization:

The learning rate schedule is initialized, adapting the learning rates for different layers of the model.
Other Initialization Tasks:

Other tasks might be performed, such as unfreezing specific layers of the model if gradual unfreezing is applied.
After the "0th epoch," the subsequent epochs involve actual weight updates based on the computed gradients during backpropagation.

In fastai, the training loop often handles these initialization tasks automatically, making it convenient for the user. The distinction of the "0th epoch" is more of a conceptual one to highlight the specific procedures that occur at the beginning of fine-tuning.







Let us build an encoder which will be specific for this task and will then fine tune again

In [12]:
files = get_text_files(path)

from concurrent.futures import ProcessPoolExecutor

def read_file(file):
    return open(file, 'r').read()

with ProcessPoolExecutor() as executor:
    texts = list(executor.map(read_file, files))

In [18]:
df = pd.DataFrame({'text': texts})

# Display the DataFrame
print(df.head())
#df.size

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      text
0   The patient comes in for a neurology consultation regarding her difficult headaches, tunnel vision, and dizziness.  transcription: PROBLEMS AND ISSUES:,1.  Headaches, nausea, and dizziness, consistent with a diagnosis of vestibular migraine, recommend amitriptyline for prophylactic treatment and Motrin for abortive treatment.,2.  Some degree of peripheral neuropathy, consistent with diabeti

In [34]:

text_block = TextBlock.from_df('text', is_lm=True, vocab=None)

# Create the TextDataLoaders with the TextBlock
dls_lm = DataBlock(blocks=text_block, get_x=ColReader('text'), splitter=RandomSplitter(valid_pct=0.1)).dataloaders(df, bs=64, seq_len=72)

# Display the vocabulary size
print("Vocabulary Size:", len(dls_lm.vocab))

Vocabulary Size: 14896


In [35]:
dls_lm.show_batch()

,text,text_
0,"xxbos xxmaj anterior cervical discectomy at xxup c5 - c6 and xxup c6 - c7 for neural decompression and anterior interbody fusion at xxup c5 - c6 and xxup c6 - c7 utilizing xxmaj bengal cages x2 . xxmaj anterior instrumentation by xxmaj uniplate construction xxup c5 , xxup c6 , and xxup c7 with intraoperative x - ray x2 . transcription : xxup preoperative xxup diagnoses : , 1 . xxmaj","xxmaj anterior cervical discectomy at xxup c5 - c6 and xxup c6 - c7 for neural decompression and anterior interbody fusion at xxup c5 - c6 and xxup c6 - c7 utilizing xxmaj bengal cages x2 . xxmaj anterior instrumentation by xxmaj uniplate construction xxup c5 , xxup c6 , and xxup c7 with intraoperative x - ray x2 . transcription : xxup preoperative xxup diagnoses : , 1 . xxmaj cervical"
1,"were avoided . xxmaj at this time , through the original skin incision , attention was directed to the first intermetatarsal space where utilizing both sharp and blunt dissection the deep transverse intermetatarsal ligament was identified . xxmaj this was then incised fully exposing the tendon and the abductor hallucis muscle . xxmaj this was then resected from his osseous attachments and a small tenotomy was performed . xxmaj at this time","avoided . xxmaj at this time , through the original skin incision , attention was directed to the first intermetatarsal space where utilizing both sharp and blunt dissection the deep transverse intermetatarsal ligament was identified . xxmaj this was then incised fully exposing the tendon and the abductor hallucis muscle . xxmaj this was then resected from his osseous attachments and a small tenotomy was performed . xxmaj at this time ,"
2,"stable condition . xxmaj chart and informed consent were all reviewed preop . xxmaj all risks , benefits , and alternatives regarding the procedure have been reviewed in detail with the patient . xxmaj she is aware of risks include but not limited to bleeding , infection , scarring , recurrence of the lesion , need for further procedures , etc . xxmaj the areas of concern were marked with the marking","condition . xxmaj chart and informed consent were all reviewed preop . xxmaj all risks , benefits , and alternatives regarding the procedure have been reviewed in detail with the patient . xxmaj she is aware of risks include but not limited to bleeding , infection , scarring , recurrence of the lesion , need for further procedures , etc . xxmaj the areas of concern were marked with the marking pen"
3,". xxmaj despite frequent decreases in her insulin regimen , she continues to have somewhat low blood glucoses , most notably in the morning when the glucoses have been in the 70s despite decreasing her xxmaj lantus insulin from around 84 units down to 60 units , which is a considerable change . xxmaj what i can not explain is why her glucoses have not really climbed at all despite the decrease","xxmaj despite frequent decreases in her insulin regimen , she continues to have somewhat low blood glucoses , most notably in the morning when the glucoses have been in the 70s despite decreasing her xxmaj lantus insulin from around 84 units down to 60 units , which is a considerable change . xxmaj what i can not explain is why her glucoses have not really climbed at all despite the decrease in"
4,"xxmaj she comes in today for reevaluation because she is flaring . xxmaj her hands are very dry , they are cracked , she has been washing with soap . xxmaj she states that the xxmaj cetaphil cleansing lotion apparently is causing some burning and pain because of the fissures in her skin . xxmaj she has been wearing some gloves also apparently . xxmaj the patient is single . xxmaj she","she comes in today for reevaluation because she is flaring . xxmaj her hands are very dry , they are cracked , she has been washing with soap . xxmaj she states that the xxmaj cetaphil cleansing lotion apparently is causing some burning and pain because of the fissures in her skin . xxmaj she has been wearing s

In [44]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [45]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time


KeyboardInterrupt: 

In [38]:
learn.save('1epoch')

Path('/content/drive/MyDrive/Kaggle_int_hc/models/1epoch.pth')

In [39]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.158102,3.014614,0.438537,20.381214,01:03
1,2.961268,2.807080,0.460778,16.561493,01:03
2,2.723581,2.611989,0.484322,13.626120,01:02
3,2.541250,2.470635,0.500622,11.829959,01:02
4,2.377009,2.358569,0.519261,10.575803,01:02
5,2.259117,2.270247,0.531877,9.681794,01:03
6,2.154806,2.210465,0.542125,9.119958,01:02
7,2.059081,2.166268,0.549344,8.725662,01:02
8,2.027591,2.147147,0.552879,8.560402,01:02
9,2.007709,2.144625,0.553506,8.538842,01:02


In [40]:
learn.save_encoder('finetuned')


In [41]:

dblock = DataBlock(blocks=(TextBlock.from_folder(path=path), CategoryBlock),
                   get_items=partial(get_text_files, folders=['train', 'valid']),
                   get_y=parent_label,
                   splitter=GrandparentSplitter(valid_name='valid'))

dls = dblock.dataloaders(source=path, bs=32)

In [46]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)


In [47]:
learn = learn.load_encoder('/content/drive/MyDrive/Kaggle_int_hc/models/finetuned')


In [48]:
learn.fit_one_cycle(1, 2e-2)


epoch,train_loss,valid_loss,accuracy,time
0,2.230592,2.126803,0.316100,00:39


In [49]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,1.950725,1.998718,0.350074,00:45


In [50]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.742350,1.903924,0.319055,01:07


In [51]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.503413,1.931049,0.290990,01:24
1,1.448197,1.860343,0.296898,01:24


Performance is even worse, will use Clinical BERT now.